<a href="https://colab.research.google.com/github/domingues100/IEEE---Water_Level/blob/main/Regress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# IMPORTS

In [ ]:
import os
from shutil import copy2
import csv
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import PIL
from tensorflow import keras
from keras import layers
from keras.layers import Dense, Flatten
from keras.models import Sequential
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import gc
import statistics
from keras.layers import Lambda
import joblib
import tensorflow_hub as hub

# **KFOLD**

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/IC_Machine_Learning/instances_default.csv')

for ind in train_data.index:
  train_data.loc[ind, 'label'] = train_data.loc[ind, 'label'] - 3

Y = train_data[['label']]

In [ ]:
num_epochs = 100

idg = ImageDataGenerator(width_shift_range=0.3,
                         height_shift_range=0.3,
                         rotation_range=0.4,
                         zoom_range=0.4,
                         fill_mode='nearest',
                         horizontal_flip = True,
                         rescale=1./255)

In [ ]:
save_dir = #save_path
image_dir = #img dir ->remember to resize to 224x224

folder_old_path = #path to the folds used in class


fold_var = 1
n = Y.size


for i in range(1,6):
  training_data = pd.read_csv(f"{folder_old_path}/training_data{i}.csv")
  validation_data = pd.read_csv(f"{folder_old_path}o/validation_data{i}.csv")

  scaler = StandardScaler()
  scaler.fit(training_data["label"].to_numpy().reshape([-1,1]))

  validation_data["normalizado"] = scaler.transform(validation_data["label"].to_numpy().reshape([-1,1]))
  training_data["normalizado"] = scaler.transform(training_data["label"].to_numpy().reshape([-1,1]))

  #data generators - training and validating
  train_data_generator = idg.flow_from_dataframe(training_data, directory = image_dir,
						             x_col = "id", y_col = "normalizado",
						             class_mode = "raw", shuffle = True, batch_size=32)

  valid_data_generator = idg.flow_from_dataframe(validation_data, directory = image_dir,
							           x_col = "id", y_col = "normalizado",
							           class_mode = "raw", shuffle = False, batch_size=32)


#Model
  base_model = keras.applications.ResNet50V2(
               include_top=False,
               weights='imagenet',
               input_shape=(224, 224, 3))

  base_model.trainable = False


  model = keras.Sequential([

    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(256, activation ='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(1, activation='linear')
  ])

  # Optimizer
  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

	# Compile Model
  model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy', 'mse'])

	# Fit Model
  history = model.fit(train_data_generator, epochs=num_epochs, validation_data=valid_data_generator) #validation_data=valid_data_generator

  # Save Model
  tf.keras.backend.clear_session()
  model.save(save_dir + f"/model{fold_var}.h5")
  joblib.dump(scaler, save_dir +"/scaler"+str(fold_var)) #saves the standart scaler to use for predictions

  fold_var += 1
  gc.collect()

# Load Model

Necessary run diretory cells and idg before



In [ ]:
from keras.models import load_model

base_model = keras.applications.ResNet50V2(
               include_top=False,
               weights='imagenet',
               input_shape=(224, 224, 3))

base_model.trainable = False

#   # Model Def
model = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(256, activation ='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(1, activation='linear')
  ])


In [ ]:
save_dir =  #save_path
image_dir = #image path ->resized to 224
folder_old_path = #path to the folds used in class

var = 1
accuracy = []
k=0


for k in range(5):
  validation_data = pd.read_csv(f"{folder_old_path}/validation_data{k+1}.csv")
  scaler =joblib.load(save_dir +"/scaler"+str(k+1))

  valid_data_generator = idg.flow_from_dataframe(validation_data, directory = image_dir,
							           x_col = "id", y_col = "label",
							           class_mode = "raw", shuffle = False, batch_size=32)



  model.load_weights(f"{save_dir}/model{k+1}.h5")
  pred = model.predict(valid_data_generator)
  pred = scaler.inverse_transform(pred)
  pred = np.squeeze(pred)

  for itens in validation_data:
    val_data = validation_data["label"] .values.tolist()


  val_data = list(map(int, val_data))

  k += 1

  dados = {"predicted_desnormalizado": pred, "labels": val_data}
  df = pd.DataFrame(dados)
  df.to_csv(save_dir+f"/df{k}.csv")